## Create SQLite Database

The goal of this notebook is to create a SQL database with all of the datasets from https://open.toronto.ca/dataset/ttc-subway-delay-data/.

- #Step 1#: we manually updated ttc-subway-delay-codes by assigning each code and description a unique code number. We also grouped the all the codes into [##] delay categories. We converted the updated file to a csv.

- #Step 2#: we created a major_events table to support our existing dataset. This table includes major sporting events, concerts, festivals and season events that took place in Toronto between 2014-2025. We used Copilot to compile this information. 

- #Step 3#: we converted our delay datatables from excel into csv. Some excel files had tabs split by month. We used the VBA code below to spit each tab into seperate csv file:
  
  Sub ExportSheetsToCSV()
                    Dim xWs As Worksheet
                    Dim xcsvFile As String
                    For Each xWs In Application.ActiveWorkbook.Worksheets
                        xWs.Copy
                        xcsvFile = Application.ThisWorkbook.Path & "\" & xWs.Name & ".csv"
                        'adds the exported files to whichever folder the current workbook is saved under
                        Application.ActiveWorkbook.SaveAs Filename:=xcsvFile, _
                        FileFormat:=xlCSV, CreateBackup:=False
                        Application.ActiveWorkbook.Saved = True
                       Application.ActiveWorkbook.Close
                   Next
               End Sub

- #Step 4#: Zip all the files together in the following location "C:\Users\aakav\dsi\ml13-ttc-delays-project\src_files\csv converted raw data.zip"

- #Step 5#: run the following scripts below

In [8]:
import zipfile
import os
import sqlite3
import pandas as pd

In [9]:
# Define Paths
zip_path = r"C:\Users\aakav\dsi\ml13-ttc-delays-project\src_files\csv converted raw data.zip"
extract_folder = r"C:\Users\aakav\dsi\ml13-ttc-delays-project\src_files\unzipped_csvs_raw_data"
db_path = r"C:\Users\aakav\dsi\ml13-ttc-delays-project\src_files\ttc_delays.db"

In [10]:
#Unzip csvs
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)
print(f"Unzipped to: {extract_folder}")


Unzipped to: C:\Users\aakav\dsi\ml13-ttc-delays-project\src_files\unzipped_csvs_raw_data


In [11]:
#Inspect column names and data types
print("\n📊 Column inspection:")
for filename in sorted(os.listdir(extract_folder)):
    file_path = os.path.join(extract_folder, filename)
    df = pd.read_csv(file_path, nrows=10)
    print(f"\n🧾 {filename}")
    print(df.dtypes)

## Brought this step in because I noticed TTC_Subway_Delay_Data_since_2025 had a unique "row count" id column which prevented us from unioning all the delay tables in SQL.
## Manually deleted this column in TTC_Subway_Delay_Data_since_2025 and re-ran this process


📊 Column inspection:

🧾 Apr_19.csv
Date         object
Time         object
Day          object
Station      object
Code         object
Min Delay     int64
Min Gap       int64
Bound        object
Line         object
Vehicle       int64
dtype: object

🧾 April_20.csv
Date         object
Time         object
Day          object
Station      object
Code         object
Min Delay     int64
Min Gap       int64
Bound        object
Line         object
Vehicle       int64
dtype: object

🧾 April_2018.csv
Date         object
Time         object
Day          object
Station      object
Code         object
Min Delay     int64
Min Gap       int64
Bound        object
Line         object
Vehicle       int64
dtype: object

🧾 April_21.csv
Date         object
Time         object
Day          object
Station      object
Code         object
Min Delay     int64
Min Gap       int64
Bound        object
Line         object
Vehicle       int64
dtype: object

🧾 August_18.csv
Date         object
Time         object
D

In [12]:
# Create SQLite DB
conn = sqlite3.connect(db_path)


In [13]:
# Import all contents in the SQLite DB
for filename in os.listdir(extract_folder):
    if filename.endswith(".csv"):
        table_name = os.path.splitext(filename)[0]
        file_path = os.path.join(extract_folder, filename)

        try:
            df = pd.read_csv(file_path)

            # Normalize dates to be the Y-M-D across all files that have it
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.strftime('%Y-%m-%d')
                df = df.dropna(subset=['Date'])
            
            # Normalize Time to HH:MM format
            if 'Time' in df.columns:
                def pad_time(t):
                    try:
                        h, m = map(int, str(t).split(':'))
                        return f"{h:02d}:{m:02d}"
                    except:
                        return None  # or np.nan if using numpy

                df['Time'] = df['Time'].apply(pad_time)
                df = df.dropna(subset=['Time'])  # optional: drop malformed times

            # Write to SQLite DB
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"Imported '{filename}' as table '{table_name}'")

        except Exception as e:
            print(f"Error importing '{filename}': {e}")

conn.close()
print(f"\n✅ All tables loaded into: {db_path}")


Imported 'April_20.csv' as table 'April_20'
Imported 'April_2018.csv' as table 'April_2018'
Imported 'April_21.csv' as table 'April_21'
Imported 'Apr_19.csv' as table 'Apr_19'
Imported 'August_18.csv' as table 'August_18'
Imported 'August_19.csv' as table 'August_19'
Imported 'August_20.csv' as table 'August_20'
Imported 'August_21.csv' as table 'August_21'
Imported 'December_19.csv' as table 'December_19'
Imported 'December_21.csv' as table 'December_21'
Imported 'Dec_2018.csv' as table 'Dec_2018'
Imported 'Dec_2020.csv' as table 'Dec_2020'
Imported 'Feb_18.csv' as table 'Feb_18'
Imported 'Feb_19.csv' as table 'Feb_19'
Imported 'Feb_20.csv' as table 'Feb_20'
Imported 'Feb_21.csv' as table 'Feb_21'
Imported 'January_21.csv' as table 'January_21'
Imported 'Jan_18.csv' as table 'Jan_18'
Imported 'Jan_19.csv' as table 'Jan_19'
Imported 'Jan_2020.csv' as table 'Jan_2020'
Imported 'July_18.csv' as table 'July_18'
Imported 'July_19.csv' as table 'July_19'
Imported 'July_20.csv' as table 'Jul

In [14]:
# Delete the unzipped raw folder
import shutil

shutil.rmtree(extract_folder)
